# AutoML Image Object Detection in pipeline

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Create a pipeline with Image Object Detection AutoML task.

**Motivations** - This notebook explains how to use Image Object Detection AutoML task inside pipeline.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [ ]:
# import required libraries
from azure.identity import DefaultAzureCredential

from azure.ai.ml import MLClient, Input, command, Output, dsl
from azure.ai.ml.automl import image_object_detection, ImageObjectDetectionSearchSpace
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.sweep import BanditPolicy, Choice, Uniform

# add environment variable to enable private preview feature
import os
os.environ["AZURE_ML_CLI_PRIVATE_FEATURES_ENABLED"] = "true"

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [ ]:
credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace = "<AML_WORKSPACE_NAME>"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)
print(ml_client)

## 1.3 Download Data

Load the 'fridge items' dataset from a JSON file and MLTable definition.

In order to generate models for computer vision, you will need to bring in labeled image data as input for model training in the form of an Azure Machine Learning MLTable. 

In this notebook, we use a toy dataset called Fridge Objects, which consists of 134 images of 4 classes of beverage container {can, carton, milk bottle, water bottle} photos taken on different backgrounds.

All images in this notebook are hosted in [this repository](https://github.com/microsoft/computervision-recipes) and are made available under the [MIT license](https://github.com/microsoft/computervision-recipes/blob/master/LICENSE).

**NOTE:** In this PRIVATE PREVIEW we're defining the MLTable in a separate folder and .YAML file.
In later versions, you'll be able to do it all in Python APIs.

In [ ]:
import os
import urllib
from zipfile import ZipFile

# download data
download_url = "https://cvbp-secondary.z19.web.core.windows.net/datasets/object_detection/odFridgeObjects.zip"
data_file = "./data/odFridgeObjects.zip"
urllib.request.urlretrieve(download_url, filename=data_file)

# extract files
with ZipFile(data_file, "r") as zip:
    print("extracting files...")
    zip.extractall(path="./data")
    print("done")
# delete zip file
os.remove(data_file)

This is a sample image from this dataset:

In [ ]:
from IPython.display import Image

sample_image = "./data/odFridgeObjects/images/31.jpg"
Image(filename=sample_image)

### Upload the images to Datastore through an AML Data asset (URI Folder)

In order to use the data for training in Azure ML, we upload it to our default Azure Blob Storage of our  Azure ML Workspace.

Reference to URI FOLDER data asset example for further details: https://github.com/Azure/azureml-examples/blob/samuel100/data-samples/sdk/assets/data/data.ipynb

In [ ]:
# Uploading image files by creating a 'data asset URI FOLDER':

from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_data = Data(
    path="./data/odFridgeObjects",
    type=AssetTypes.URI_FOLDER,
    description="Fridge-items images Object detection",
    name="fridge-items-images-object-detection",
)

uri_folder_data_asset = ml_client.data.create_or_update(my_data)

print(uri_folder_data_asset)
print("")
print("Path to folder in Blob Storage:")
print(uri_folder_data_asset.path)

### Convert the downloaded data to JSONL

In order to use this data to create an AzureML MLTable, we first need to convert it to the required JSONL format. 

The following script is creating two .jsonl files (one for training and one for validation) in the parent folder of the dataset. The train / validation ratio corresponds to 20% of the data going into the validation file.

In [ ]:
import json
import os
import xml.etree.ElementTree as ET

src_images = "./data/odFridgeObjects/"

# We'll copy each JSONL file within its related MLTable folder
training_mltable_path = "./data/training-mltable-folder/"
validation_mltable_path = "./data/validation-mltable-folder/"

train_validation_ratio = 5

# Path to the training and validation files
train_annotations_file = os.path.join(training_mltable_path, "train_annotations.jsonl")
validation_annotations_file = os.path.join(
    validation_mltable_path, "validation_annotations.jsonl"
)

# Baseline of json line dictionary
json_line_sample = {
    "image_url": uri_folder_data_asset.path,
    "image_details": {"format": None, "width": None, "height": None},
    "label": [],
}

# Path to the annotations
annotations_folder = os.path.join(src_images, "annotations")

# Read each annotation and convert it to jsonl line
with open(train_annotations_file, "w") as train_f:
    with open(validation_annotations_file, "w") as validation_f:
        for i, filename in enumerate(os.listdir(annotations_folder)):
            if filename.endswith(".xml"):
                print("Parsing " + os.path.join(src_images, filename))

                root = ET.parse(os.path.join(annotations_folder, filename)).getroot()

                width = int(root.find("size/width").text)
                height = int(root.find("size/height").text)

                labels = []
                for object in root.findall("object"):
                    name = object.find("name").text
                    xmin = object.find("bndbox/xmin").text
                    ymin = object.find("bndbox/ymin").text
                    xmax = object.find("bndbox/xmax").text
                    ymax = object.find("bndbox/ymax").text
                    isCrowd = int(object.find("difficult").text)
                    labels.append(
                        {
                            "label": name,
                            "topX": float(xmin) / width,
                            "topY": float(ymin) / height,
                            "bottomX": float(xmax) / width,
                            "bottomY": float(ymax) / height,
                            "isCrowd": isCrowd,
                        }
                    )
                # build the jsonl file
                image_filename = root.find("filename").text
                _, file_extension = os.path.splitext(image_filename)
                json_line = dict(json_line_sample)
                json_line["image_url"] = (
                    json_line["image_url"] + "images/" + image_filename
                )
                json_line["image_details"]["format"] = file_extension[1:]
                json_line["image_details"]["width"] = width
                json_line["image_details"]["height"] = height
                json_line["label"] = labels

                if i % train_validation_ratio == 0:
                    # validation annotation
                    validation_f.write(json.dumps(json_line) + "\n")
                else:
                    # train annotation
                    train_f.write(json.dumps(json_line) + "\n")
            else:
                print("Skipping unknown file: {}".format(filename))

# 2. Retraining pipeline job with Image Object Detection task

## 2.1 Load model register component

In [ ]:
from azure.ai.ml import load_component
parent_dir = "."
register_model = load_component(path=parent_dir + "/register_model_pipeline.yml")

## 2.2 Build retraining pipeline

In [ ]:
# note that the used docker image doesn't suit for all size of gpu compute. Please use the following command to create gpu compute if experiment failed
# !az ml compute create -n gpu-cluster --type amlcompute --min-instances 0 --max-instances 4 --size Standard_NC12

In [ ]:
# Define pipeline
@dsl.pipeline(
    name="AutoML Image Object Detection Retraining Pipeline", default_compute_target="gpu-cluster"
    )
def automl_image_object_detection_retraining(
    image_object_detection_train_data,
    image_object_detection_validation_data
):
    # define the automl image_object-detection task with automl function
    image_object_detection_node = image_object_detection(
        training_data=image_object_detection_train_data,
        validation_data=image_object_detection_validation_data,
        target_column_name="label",
        primary_metric="MeanAveragePrecision",
        # currently need to specify outputs "mlflow_model" explictly to reference it in following nodes 
        outputs={"best_model": Output(type="mlflow_model")},
    )
    image_object_detection_node.set_limits(timeout_minutes=180)

    image_object_detection_node.extend_search_space(
        [
            ImageObjectDetectionSearchSpace(
                model_name=Choice(["fasterrcnn_resnet50_fpn"]),
                learning_rate=Uniform(0.0001, 0.001),
                optimizer=Choice(["sgd", "adam", "adamw"]),
                min_size=Choice([600, 800]),  # model-specific
            ),
        ]
    )
    
    image_object_detection_node.set_image_model(early_stopping=True, evaluation_frequency=1)
    image_object_detection_node.set_sweep(
        max_trials=10,
        max_concurrent_trials=2,
        sampling_algorithm="Random",
        early_termination=BanditPolicy(evaluation_interval=2, slack_factor=0.2, delay_evaluation=6),
    )
    
    # provide model name to register the model
    model_register_job = register_model(
        automl_best_run=image_object_detection_node.outputs.best_model,
        primary_metric= image_object_detection_node.primary_metric.value,
        training_mltable = image_object_detection_train_data,
        model_name = "automl_imagde_object_detection_model",
        register_always = False
    )


data_folder = "./data"
pipeline = automl_image_object_detection_retraining(
    image_object_detection_train_data=Input(path=f"{data_folder}/training-mltable-folder/", type="mltable"),
    image_object_detection_validation_data=Input(path=f"{data_folder}/validation-mltable-folder/", type="mltable"),
)

# 2.2 Submit pipeline job

In [ ]:
# submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline, experiment_name="retraining_pipeline"
)
pipeline_job

In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

# Next Steps
You can see further examples of running a pipeline job [here](../)